# California 2Monthly Prediction

## Imports and Functions

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor


from sklearn.svm import SVR
import os
import numpy as np
from xgboost import XGBRegressor
import sys

In [2]:
# Add the folder to the Python path

os.chdir("../")
# change working directory to project's root path
print(os.getcwd())

FIRST_YEAR= 1972
FREQUENCY= "2monthly"

c:\Users\marti\Desktop\data\hw_extra


In [3]:
folder_path = os.path.abspath("functions/") #INPUT_PATH)#'path_to_your_folder')  # Replace with the actual folder path
sys.path.insert(0, folder_path)

from Predictions import (
    PredictionExperiment,
    PredictionModel
)

In [4]:
region="california"
metadata = pd.read_csv(f"data/new_features/{region}/metadata.csv")
metadata = metadata[metadata["frequency"]==FREQUENCY]
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
60,ec28ad68,predictor_ec28ad68_1.parquet,0,5,NaN,1,2monthly,1,ps,NaN,True
61,ec28ad68,predictor_ec28ad68_2.parquet,0,5,NaN,1,2monthly,2,ps,NaN,True
62,ec28ad68,predictor_ec28ad68_3.parquet,0,5,NaN,1,2monthly,3,ps,NaN,True
63,ec28ad68,predictor_ec28ad68_4.parquet,0,5,NaN,1,2monthly,4,ps,NaN,True
64,ec28ad68,predictor_ec28ad68_5.parquet,0,5,NaN,1,2monthly,5,ps,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
1495,31df530c,predictor_31df530c_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,True
1496,31df530c,predictor_31df530c_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,True
1497,31df530c,predictor_31df530c_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,True
1498,31df530c,predictor_31df530c_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,True


In [5]:
last_id = "8684951e"
index = metadata[metadata["id"]==last_id]
metadata = metadata.iloc[list(index.index)[-1]+1:]
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
780,67d93c43,predictor_67d93c43_1.parquet,0,5,NaN,1,2monthly,1,ps,PDO-ONI-SAM,True
781,67d93c43,predictor_67d93c43_2.parquet,0,5,NaN,1,2monthly,2,ps,PDO-ONI-SAM,True
782,67d93c43,predictor_67d93c43_3.parquet,0,5,NaN,1,2monthly,3,ps,PDO-ONI-SAM,True
783,67d93c43,predictor_67d93c43_4.parquet,0,5,NaN,1,2monthly,4,ps,PDO-ONI-SAM,True
784,67d93c43,predictor_67d93c43_5.parquet,0,5,NaN,1,2monthly,5,ps,PDO-ONI-SAM,True
...,...,...,...,...,...,...,...,...,...,...,...
1495,31df530c,predictor_31df530c_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,True
1496,31df530c,predictor_31df530c_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,True
1497,31df530c,predictor_31df530c_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,True
1498,31df530c,predictor_31df530c_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,True


In [6]:
kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
regressors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=5), RandomForestRegressor(random_state=42, n_estimators=10), SVR(kernel='rbf'),
                XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1),XGBRegressor(random_state=42, n_estimators=15, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
name_regressors = ["Linear", "RF5", "RF10", "SVR-rbf", "XGB10", "XGB15", "GPR"]
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]

In [7]:
id_experiments = metadata["id"].unique()
for id in id_experiments:
    data = {i: pd.read_parquet(f"data/new_features/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
    experiment_1 = PredictionExperiment(data, indices_of_interest, regressors, name_regressors, 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("r2", stage="CV", show=False)
    experiment_1.get_metrics("mape", stage="CV", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/{region}_results/results_sst.csv")

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warn

# Chile 2Monthly Prediction

In [4]:
region="chile"

In [5]:
metadata = pd.read_csv(f"data/new_features/{region}/metadata.csv")
metadata = metadata[metadata["frequency"]==FREQUENCY]
metadata.reset_index(drop=True,inplace=True)
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
0,515c5f16,predictor_515c5f16_1.parquet,0,5,NaN,1,2monthly,1,ps,NaN,True
1,515c5f16,predictor_515c5f16_2.parquet,0,5,NaN,1,2monthly,2,ps,NaN,True
2,515c5f16,predictor_515c5f16_3.parquet,0,5,NaN,1,2monthly,3,ps,NaN,True
3,515c5f16,predictor_515c5f16_4.parquet,0,5,NaN,1,2monthly,4,ps,NaN,True
4,515c5f16,predictor_515c5f16_5.parquet,0,5,NaN,1,2monthly,5,ps,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
1435,cf9ff510,predictor_cf9ff510_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,True
1436,cf9ff510,predictor_cf9ff510_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,True
1437,cf9ff510,predictor_cf9ff510_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,True
1438,cf9ff510,predictor_cf9ff510_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,True


In [6]:
last_id = "431ba565"
index = metadata[metadata["id"]==last_id]
metadata = metadata.iloc[list(index.index)[-1]+1:]
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
348,24d0ade8,predictor_24d0ade8_1.parquet,0,30,0.05,2,2monthly,1,ps,NaN,True
349,24d0ade8,predictor_24d0ade8_2.parquet,0,30,0.05,2,2monthly,2,ps,NaN,True
350,24d0ade8,predictor_24d0ade8_3.parquet,0,30,0.05,2,2monthly,3,ps,NaN,True
351,24d0ade8,predictor_24d0ade8_4.parquet,0,30,0.05,2,2monthly,4,ps,NaN,True
352,24d0ade8,predictor_24d0ade8_5.parquet,0,30,0.05,2,2monthly,5,ps,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
1435,cf9ff510,predictor_cf9ff510_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,True
1436,cf9ff510,predictor_cf9ff510_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,True
1437,cf9ff510,predictor_cf9ff510_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,True
1438,cf9ff510,predictor_cf9ff510_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,True


In [7]:
kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
regressors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=5), RandomForestRegressor(random_state=42, n_estimators=10), SVR(kernel='rbf'),
                XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1),XGBRegressor(random_state=42, n_estimators=15, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
name_regressors = ["Linear", "RF5", "RF10", "SVR-rbf", "XGB10", "XGB15", "GPR"]
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]

In [8]:
id_experiments = metadata["id"].unique()
for id in id_experiments:
    data = {i: pd.read_parquet(f"data/new_features/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
    experiment_1 = PredictionExperiment(data, indices_of_interest, regressors, name_regressors, 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("r2", stage="CV", show=False)
    experiment_1.get_metrics("mape", stage="CV", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/{region}_results/results_sst.csv")

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c